# Tải video YouTube kèm phụ đề (yt-dlp)

Mục tiêu: tạo công cụ tải video từ YouTube, kèm phụ đề (subtitles hoặc auto-captions), lưu vào thư mục dự án `API/AI/data/videos/`.

Thành phần:
- Cài đặt `yt-dlp` và kiểm tra `ffmpeg`.
- Hàm `download_youtube_with_subtitles(...)` tải video/playlist, ưu tiên MP4 và thu thập phụ đề.
- Tuỳ chọn ngôn ngữ phụ đề (ví dụ `vie`, `eng`), định dạng `.srt`.
- Log tiến độ và trả về danh sách đường dẫn tệp đã tải.

**Lưu ý**: `ffmpeg` cần có trong PATH để hợp nhất video/audio và chuyển định dạng phụ đề.

In [1]:

# Cài đặt thư viện cần thiết
# - yt-dlp: công cụ tải YouTube
# - pathlib: làm việc với đường dẫn
# Gợi ý cài ffmpeg trước (Windows có thể dùng winget hoặc tải binary).

# !pip install --upgrade yt-dlp

import os, sys, shutil
from pathlib import Path

# Kiểm tra ffmpeg đã có trong PATH hay chưa
# Nếu None -> cần cài ffmpeg để ghép video/audio và chuyển phụ đề
ffmpeg_path = shutil.which("ffmpeg")
print("ffmpeg:", ffmpeg_path if ffmpeg_path else "CHƯA PHÁT HIỆN (cần cài)")

# Tạo thư mục đích theo chuẩn OLMS
data_root = Path("API/AI/data/videos")
data_root.mkdir(parents=True, exist_ok=True)
print("Thư mục lưu:", data_root.resolve())


   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ------ --------------------------------- 0.5/3.2 MB 4.2 MB/s eta 0:00:01
   ------------------- -------------------- 1.6/3.2 MB 4.7 MB/s eta 0:00:01
   ----------------------------------- ---- 2.9/3.2 MB 5.1 MB/s eta 0:00:01
   ---------------------------------------- 3.2/3.2 MB 4.5 MB/s eta 0:00:00
ffmpeg: D:\ProgramFiles\ffmpeg-8.0-essentials_build\bin\ffmpeg.EXE
Thư mục lưu: D:\Projects\Basic_Project_CSharp\Online_Management_System - Copy\API\AI\API\AI\data\videos



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:

from typing import List, Dict, Any, Optional
from yt_dlp import YoutubeDL

def download_youtube_with_subtitles(
    url: str,
    out_dir: str = "API/AI/data/videos",
    prefer_mp4: bool = True,
    subtitles_langs: Optional[List[str]] = None,
    use_auto_captions: bool = True,
    subtitle_format: str = "srt"
) -> List[str]:
    """
    Tải video/playlist YouTube về thư mục out_dir và tải phụ đề nếu có.
    - Ưu tiên định dạng MP4; nếu không có thì lấy best và hợp nhất.
    - Phụ đề: ưu tiên ngôn ngữ chỉ định (subtitles_langs). Nếu không có, có thể cho phép auto-captions.
    - Định dạng phụ đề: srt (khuyến nghị).

    Trả về: danh sách đường dẫn file video/subtitle sau khi tải.
    """

    # Chuẩn hoá danh sách ngôn ngữ phụ đề; ví dụ ["vi","en"] hoặc ["vie","eng"]
    # yt-dlp dùng mã ngôn ngữ theo YouTube; "vi" và "vie" thường tương đương, "en" và "eng" tương đương
    if subtitles_langs is None:
        subtitles_langs = ["vi", "vie", "en", "eng"]

    out_dir_path = Path(out_dir)
    out_dir_path.mkdir(parents=True, exist_ok=True)

    # Mẫu tên tệp đầu ra: <thư_mục>/<video_id>-<tiêu_đề>.<ext>
    # Dùng video_id để tránh trùng tên
    out_tmpl = str(out_dir_path / "%(id)s-%(title)s.%(ext)s")

    # Chọn format ưu tiên mp4
    if prefer_mp4:
        # best mp4 video + best m4a audio nếu có, fallback về best mp4, cuối cùng là best bất kỳ
        fmt = "bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best"
        merge_to = "mp4"
    else:
        fmt = "bv*+ba/best"
        merge_to = "mp4"

    # Danh sách tệp đã tải (video + phụ đề)
    downloaded_files: List[str] = []

    # Hook theo dõi tiến độ; in phần trăm và tốc độ tải
    def _progress_hook(d: Dict[str, Any]):
        # Trạng thái "downloading": in tiến độ
        if d.get("status") == "downloading":
            percent = d.get("_percent_str", "").strip()
            speed = d.get("speed", None)
            speed_str = f"{speed/1024/1024:.2f} MB/s" if speed else "? MB/s"
            fname = d.get("filename", "")
            print(f"[DL] {percent} {speed_str}  {fname}", end="\r")
        # Trạng thái "finished": báo hợp nhất (nếu có)
        elif d.get("status") == "finished":
            fname = d.get("filename", "")
            print(f"\n[MERGE] Đã tải xong phân đoạn: {fname} -> hợp nhất nếu cần...")

    # Post-processor tự thu thập đường dẫn tệp đầu ra cuối cùng
    class _CollectorPP:
        def run(self, info):
            # info['filepath'] có thể chứa đường dẫn file video sau hợp nhất
            fp = info.get("filepath")
            if fp and Path(fp).exists():
                downloaded_files.append(fp)
            # Nếu có phụ đề đã lưu kèm, thêm vào danh sách
            # yt-dlp lưu phụ đề cùng tên gốc nhưng .srt/.vtt trong cùng thư mục
            # Thu thập bằng cách quét thư mục theo video id
            vid = info.get("id")
            if vid:
                for p in Path(out_dir).glob(f"{vid}-*.{subtitle_format}"):
                    if str(p) not in downloaded_files:
                        downloaded_files.append(str(p))
            return [], info

    # Tuỳ chọn phụ đề trong yt-dlp
    # - writesubtitles: tải phụ đề do người up (nếu có)
    # - writeautomaticsub: tải auto-captions (nếu có) khi writesubtitles không có
    # - sublangs: danh sách ngôn ngữ ưu tiên
    # - subtitlesformat: định dạng đích (vtt hoặc srt). Để srt, cần FFmpegSubtitlesConvertor.
    ydl_opts = {
        "outtmpl": out_tmpl,
        "format": fmt,
        "merge_output_format": merge_to,
        "noplaylist": False,                 # chấp nhận playlist
        "ignoreerrors": True,                # không dừng nếu một video trong playlist lỗi
        "progress_hooks": [_progress_hook],
        "quiet": False,
        "no_warnings": True,

        # Cấu hình phụ đề
        "writesubtitles": True,              # bật tải phụ đề do uploader cung cấp
        "writeautomaticsub": use_auto_captions,  # bật auto-captions nếu cần
        "subtitleslangs": subtitles_langs,   # ưu tiên ngôn ngữ
        "subtitlesformat": subtitle_format,  # "srt" hoặc "vtt"

        # Chuyển phụ đề sang định dạng đích (cần ffmpeg cho srt)
        "postprocessors": [
            {
                "key": "FFmpegSubtitlesConvertor",
                "format": subtitle_format
            }
        ]
    }

    with YoutubeDL(ydl_opts) as ydl:
        # Gắn collector để lấy danh sách file đã sinh ra
        ydl.add_post_processor(_CollectorPP(), when="post_process")
        # Thực thi tải
        ydl.download([url])

    # Loại trùng
    uniq = []
    seen = set()
    for f in downloaded_files:
        if f not in seen:
            seen.add(f)
            uniq.append(f)

    print(f"\n[OK] Tổng số tệp đã ghi: {len(uniq)}")
    for f in uniq:
        print(" -", f)
    return uniq


In [3]:

# Ví dụ sử dụng:
# - Tải 1 video đơn lẻ và phụ đề tiếng Việt/Anh nếu có.
# - Lưu về API/AI/data/videos
# - Trả về danh sách đường dẫn tệp (video .mp4 và phụ đề .srt)

# url_demo có thể thay bằng video bất kỳ cần thử
url_demo = "https://youtu.be/AWWtNNIfAOo?si=yEItsV2Z1mcHH64m"

files = download_youtube_with_subtitles(
    url=url_demo,
    out_dir="API/AI/data/videos",
    prefer_mp4=True,
    subtitles_langs=["vi","vie","en","eng"],
    use_auto_captions=True,
    subtitle_format="srt"
)

# In danh sách tệp đã lưu
from pprint import pprint
pprint(files)


AttributeError: '_CollectorPP' object has no attribute 'set_downloader'

In [ ]:

# Liệt kê toàn bộ tệp trong thư mục đích để kiểm tra
from pathlib import Path
from datetime import datetime

root = Path("API/AI/data/videos")
rows = []
for p in sorted(root.glob("*")):
    if p.is_file():
        st = p.stat()
        rows.append({
            "name": p.name,
            "size_MB": round(st.st_size/1024/1024, 2),
            "modified": datetime.fromtimestamp(st.st_mtime).strftime("%Y-%m-%d %H:%M:%S"),
            "path": str(p.resolve())
        })

import pandas as pd
pd.DataFrame(rows).head(50)
